In [ ]:
import numpy as np
from numpy import random as rd
import time
from solvers_listos import *
from funciones import *

In [1]:
%run politica_R.ipynb
%run politica_P.ipynb
# hay que ver qué onda este print

{0: 0, 1: 91, 2: 27, 3: 11, 4: 63, 5: 69, 6: 67, 7: 91, 8: 46, 9: 73, 10: 66}


In [2]:
G, ubis, cap_tpte, info_locales = crear_grafo_inicial(archivo= 'IRP1.xlsx' ,plot=False)
historia = simular_demanda_previa(G, dist = 'n', T=1000) 

In [3]:
rutas_R, perdidas_R, inventarios_R = simular_ejecucion_R(grafo_inicial = G, T = 60)

Inventario inicial: 
N_0 1428
N_1 87
N_2 27
N_3 20
N_4 55
N_5 142
N_6 69
N_7 158
N_8 86
N_9 77
N_10 126


Ruta 0:  ['N_0', 'N_6', 'N_0']
Ruta 1:  ['N_0', 'N_4', 'N_9', 'N_2', 'N_1', 'N_3', 'N_6', 'N_8', 'N_0']
Ruta 2:  ['N_0', 'N_10', 'N_2', 'N_7', 'N_5', 'N_0']
Ruta 3:  ['N_0', 'N_4', 'N_9', 'N_1', 'N_6', 'N_8', 'N_0']
Ruta 4:  ['N_0', 'N_4', 'N_9', 'N_2', 'N_1', 'N_3', 'N_7', 'N_5', 'N_0']
Ruta 5:  ['N_0', 'N_10', 'N_9', 'N_2', 'N_1', 'N_6', 'N_0']
Ruta 6:  ['N_0', 'N_4', 'N_5', 'N_8', 'N_6', 'N_2', 'N_0']
Ruta 7:  ['N_0', 'N_9', 'N_2', 'N_1', 'N_3', 'N_7', 'N_6', 'N_0']
Ruta 8:  ['N_0', 'N_4', 'N_10', 'N_9', 'N_1', 'N_8', 'N_0']
Ruta 9:  ['N_0', 'N_2', 'N_7', 'N_6', 'N_5', 'N_0']
Ruta 10:  ['N_0', 'N_4', 'N_9', 'N_2', 'N_1', 'N_3', 'N_6', 'N_8', 'N_0']
Ruta 11:  ['N_0', 'N_4', 'N_10', 'N_9', 'N_2', 'N_1', 'N_7', 'N_5', 'N_0']
Ruta 12:  ['N_0', 'N_4', 'N_9', 'N_2', 'N_1', 'N_6', 'N_0']
Ruta 13:  ['N_0', 'N_5', 'N_8', 'N_6', 'N_3', 'N_0']
Ruta 14:  ['N_0', 'N_4', 'N_10', 'N_9', 'N_2',

In [6]:
rutas_P, perdidas_P, inventarios_P = simular_ejecucion_P_LS(grafo_inicial = G, dem_historico= historia, T = 60, F = 5, cap = 871)

F = 5, Demanda perdida total: 12603.256328435977 | Demanda perdida promedio: 210.0542721405996


In [ ]:
mis_rutas
mis_rutas_t = {t : str(mis_rutas[t]) for t in mis_rutas.keys()}
df_rutas = pd.DataFrame.from_dict(mis_rutas_t, orient='index', columns=['Ruta'])
df_rutas.value_counts()



N = len(G.nodes())-1
df = generar_df(mis_rutas, N)
df['sum'] = df.sum(axis=1)


In [ ]:
df_nodos = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
df_nodos = df_nodos.iloc[1:]
df_nodos['frec'] = (df.describe()).T['mean']
dem_historico_media = {nodo: np.mean(historia[nodo]) for nodo in historia.keys()}
df_nodos['dem_mean'] = dem_historico_media.values()
df_nodos

In [ ]:
plt.figure(figsize=(21, 7))
plt.plot(df['sum'], '-', color='black')
plt.xlabel('Días')
plt.ylabel('Locales visitados')
plt.title('Locales visitados por día')
plt.show()

In [ ]:
plotear_tablero_visitas(df, guardar = True, nombre = 'plots//tablero_PRO_n30_orig.png')

In [ ]:
cantidad_06 = len(df_nodos[df_nodos['frec'] >= 0.5])
cantidad_04 = len(df_nodos[df_nodos['frec'] < 0.5])

print (f'Locales visitados más de 50% de los días: {cantidad_06}, que corresponde a un {cantidad_06/N*100}%')
print (f'Locales visitados menos de 50% de los días: {cantidad_04}, que corresponde a un {cantidad_04/N*100}%')

In [ ]:
datos = dispersion_intervalos(df)
        
df_nodos['mean'] = datos['mean']
df_nodos['std'] = datos['std']
df_nodos['porcentaje demanda'] = df_nodos['dem_mean'] / df_nodos['Up']
df_nodos['porcentaje dem. inicial'] = df_nodos['Prod'] / df_nodos['Up']
df_nodos_corr = df_nodos[['Inv','Up','Low','Prod','h','frec','mean','std','dem_mean','porcentaje demanda','porcentaje dem. inicial']]
df_nodos

In [ ]:
df_nodos_corr.corr()